In [2]:
pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import surprise 

print(surprise.__version__)

1.1.4


1. Import Data

In [115]:
import os
import pandas as pd
csv_path = './amazon.csv'

amazon_df = pd.read_csv(csv_path)
print(amazon_df.shape)

(1465, 16)


2. EDA

In [116]:
amazon_df.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...


In [117]:
amazon_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

In [118]:
amazon_df.describe()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
count,1465,1465,1465,1465,1465,1465,1465,1463,1465,1465,1465,1465,1465,1465,1465,1465
unique,1351,1337,211,550,449,92,28,1143,1293,1194,1194,1194,1194,1212,1412,1465
top,B07JW9H4J1,"Fire-Boltt Ninja Call Pro Plus 1.83"" Smart Wat...",Computers&Accessories|Accessories&Peripherals|...,₹199,₹999,50%,4.1,"9,378",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AHIKJUDTVJ4T6DV6IUGFYZ5LXMPA,AE55KTFVNXYFD5FPY...","$@|\|TO$|-|,Sethu madhav,Akash Thakur,Burger P...","R3F4T5TRYPTMIG,R3DQIEC603E7AY,R1O4Z15FD40PV5,R...","Worked on iPhone 7 and didn’t work on XR,Good ...","I am not big on camera usage, personally. I wa...",https://m.media-amazon.com/images/I/413sCRKobN...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
freq,3,5,233,53,120,56,244,9,6,10,10,10,10,8,3,1


2-1. Extract Data need only for ML with Matrix Completion

In [119]:
# Matrix Completion
from itertools import product

# One userId cell has multiple userid, seperate and make dataframe
def explode_user_ids(df):
    df = df.copy()
    df['user_id'] = df['user_id'].astype(str)
    df['user_id'] = df['user_id'].str.split(',')
    return df.explode('user_id').reset_index(drop=True)


amazon_recommender_df = explode_user_ids(amazon_df)

In [120]:
amazon_recommender_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11503 entries, 0 to 11502
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           11503 non-null  object
 1   product_name         11503 non-null  object
 2   category             11503 non-null  object
 3   discounted_price     11503 non-null  object
 4   actual_price         11503 non-null  object
 5   discount_percentage  11503 non-null  object
 6   rating               11503 non-null  object
 7   rating_count         11501 non-null  object
 8   about_product        11503 non-null  object
 9   user_id              11503 non-null  object
 10  user_name            11503 non-null  object
 11  review_id            11503 non-null  object
 12  review_title         11503 non-null  object
 13  review_content       11503 non-null  object
 14  img_link             11503 non-null  object
 15  product_link         11503 non-null  object
dtypes: o

In [121]:
users = amazon_recommender_df['user_id'].unique()
items = amazon_recommender_df['product_id'].unique()
pairs = pd.DataFrame(product(users, items), columns=['user_id','product_id'])

cleaned_df = amazon_recommender_df.drop_duplicates(subset=['user_id', 'product_id'])


amazon_recomm_df = pairs.merge(
    cleaned_df[['user_id', 'product_id', 'rating']],
    on=['user_id', 'product_id'], 
    how='left'
)

product_map = amazon_recommender_df[['product_id','product_name']].drop_duplicates()
amazon_recomm_df = amazon_recomm_df.merge(product_map, on = 'product_id', how = 'left')

In [122]:
print(amazon_recomm_df.head(10))
amazon_recomm_df.shape

                        user_id  product_id rating  \
0  AG3D6O4STAQKAY2UVGEUV46KN35Q  B07JW9H4J1    4.2   
1  AG3D6O4STAQKAY2UVGEUV46KN35Q  B098NS6PVG    NaN   
2  AG3D6O4STAQKAY2UVGEUV46KN35Q  B096MSW6CT    NaN   
3  AG3D6O4STAQKAY2UVGEUV46KN35Q  B08HDJ86NZ    NaN   
4  AG3D6O4STAQKAY2UVGEUV46KN35Q  B08CF3B7N1    NaN   
5  AG3D6O4STAQKAY2UVGEUV46KN35Q  B08Y1TFSP6    NaN   
6  AG3D6O4STAQKAY2UVGEUV46KN35Q  B08WRWPM22    NaN   
7  AG3D6O4STAQKAY2UVGEUV46KN35Q  B08DDRGWTJ    NaN   
8  AG3D6O4STAQKAY2UVGEUV46KN35Q  B008IFXQFU    NaN   
9  AG3D6O4STAQKAY2UVGEUV46KN35Q  B082LZGK39    NaN   

                                        product_name  
0  Wayona Nylon Braided USB to Lightning Fast Cha...  
1  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...  
2  Sounce Fast Phone Charging Cable & Data Sync U...  
3  boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...  
4  Portronics Konnect L 1.2M Fast Charging 3A 8 P...  
5  pTron Solero TB301 3A Type-C Data and Fast Cha...  
6  boAt Micro USB 55

(12226550, 4)

2-2. Changing Type of Data and other value about rating

In [123]:
amazon_recomm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12226550 entries, 0 to 12226549
Data columns (total 4 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   user_id       object
 1   product_id    object
 2   rating        object
 3   product_name  object
dtypes: object(4)
memory usage: 373.1+ MB


In [130]:
import numpy as np
amazon_recomm_df[amazon_recomm_df['rating']=='|'].value_counts()

user_id                       product_id  rating  product_name                                                                                                                                               
AE3MSW6H3AL6F3ZGR5LCN5AHJO6A  B08L12N5H1  |       Eureka Forbes car Vac 100 Watts Powerful Suction Vacuum Cleaner with Washable HEPA Filter, 3 Accessories,Compact,Light Weight & Easy to use (Black and Red)    1
AER2XFSWNN4LAUCJ55IY5SOMF7WA  B08L12N5H1  |       Eureka Forbes car Vac 100 Watts Powerful Suction Vacuum Cleaner with Washable HEPA Filter, 3 Accessories,Compact,Light Weight & Easy to use (Black and Red)    1
AFLOAOURRZZZGFBF7F6IKGXRB6NQ  B08L12N5H1  |       Eureka Forbes car Vac 100 Watts Powerful Suction Vacuum Cleaner with Washable HEPA Filter, 3 Accessories,Compact,Light Weight & Easy to use (Black and Red)    1
AG5OL5WIIPJBY25HISJLM5K2UBTQ  B08L12N5H1  |       Eureka Forbes car Vac 100 Watts Powerful Suction Vacuum Cleaner with Washable HEPA Filter, 3 Accessories,Compac

In [131]:
amazon_recomm_df['rating'] = amazon_recomm_df['rating'].replace('|', np.nan)
amazon_recomm_df['rating'] = amazon_recomm_df['rating'].astype(float)

In [132]:
amazon_recomm_df['rating'].isnull().sum() # -> 1

12215954

In [133]:
amazon_recomm_df[amazon_recomm_df['rating']=='|'].value_counts()

Series([], Name: count, dtype: int64)

In [134]:
amazon_recomm_df.isnull().sum()

user_id                0
product_id             0
rating          12215954
product_name           0
dtype: int64

2-3. Check duplications

In [140]:
print(amazon_recomm_df[amazon_recomm_df.duplicated()].value_counts())

Series([], Name: count, dtype: int64)


2-4. Check Correlations

In [141]:
import plotly.express as px
corr = amazon_recomm_df.corr(numeric_only=True)
print(corr)

        rating
rating     1.0


2-5. Check 'rating'

In [142]:
amazon_recomm_df['rating'].describe()

count    10596.000000
mean         4.093196
std          0.287076
min          2.000000
25%          3.900000
50%          4.100000
75%          4.300000
max          5.000000
Name: rating, dtype: float64

3. Recommedation with Surprise

In [143]:
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV

# Data Loading
train_df = amazon_recomm_df[amazon_recomm_df['rating'].notnull()]

reader = Reader(rating_scale=(2, 5))
data = Dataset.load_from_df(train_df[['user_id', 'product_id', 'rating']], reader)

# Data Split
trainset, testset = train_test_split(data, test_size = 0.3, random_state=0)

# SVD Algorithm
algo = SVD(random_state=0)

# HyperParameter
param_grid = {'n_epochs' : [5,10,20,30], 'n_factors' : [50,100]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv = 3)
gs.fit(data)

best_algo = gs.best_estimator['rmse']
print("Best RMSE params:", gs.best_params['rmse'])
best_algo.fit(trainset)

# Cross-validate
cross_validate(best_algo, data, measures=['RMSE','MAE'], cv = 5)
predictions = best_algo.test(testset)

Best RMSE params: {'n_epochs': 30, 'n_factors': 50}


In [145]:
uid = 'AE3O6366WGEQAANKJ76QETTUQQTQ'
iid = 'B08L12N5H1'

pred = best_algo.predict(uid, iid, verbose=True)
print(pred)

user: AE3O6366WGEQAANKJ76QETTUQQTQ item: B08L12N5H1 r_ui = None   est = 4.14   {'was_impossible': False}
user: AE3O6366WGEQAANKJ76QETTUQQTQ item: B08L12N5H1 r_ui = None   est = 4.14   {'was_impossible': False}


4. Personalized Recommendation Program

In [147]:
def un_rating_product(user_id):
    unrating_product = amazon_recomm_df[(amazon_recomm_df['user_id'] == user_id) & 
        (amazon_recomm_df['rating'].isnull())
    ]['product_id'].unique().tolist()
    return unrating_product

def recomm_movie(algo, user_id, unrating_product, top_n = 10):
    predictions = [best_algo.predict(str(user_id), str(product_id)) for product_id in unrating_product]

    predictions.sort(key=lambda x: x.est, reverse=True)
    top_predictions = predictions[:top_n]

    top_pro_id = [pred.iid for pred in top_predictions]
    top_pro_rating = [pred.est for pred in top_predictions]
    
    product_map = amazon_recommender_df[['product_id', 'product_name']].drop_duplicates()
    top_pro_name = product_map[product_map['product_id'].isin(top_pro_id)].set_index('product_id').loc[top_pro_id]['product_name'].tolist()


    top_pro = list(zip(top_pro_id, top_pro_name, top_pro_rating))
    return top_pro

def recommendation():
    try:
        user_id = input("Please input your id : ").strip()

        if user_id not in amazon_recomm_df['user_id'].unique():
            print("You are not our member, Please sign in!")
            return 
            
        unrated_products = un_rating_product(user_id)

        if not unrated_products:
            print('You have already commented in all our products. Thank you!')
            return 
            
        result = recomm_movie(best_algo, user_id, unrated_products, top_n=10)

        print(f'Top10 what you will love!')
        for idx,(pid, name, rating) in enumerate(result,1):
            print(f'{idx}, {name}, (Expected Rating:{rating:.2f})')

    except KeyboardInterrupt:
        print("Because of keyboardInterrupt, program will be shutting down")

    except Exception as e:
        print("ERROR", e)

In [148]:
print(recommendation())

Please input your id :  AE3O6366WGEQAANKJ76QETTUQQTQ


Top10 what you will love!
1, Amazon Basics Wireless Mouse | 2.4 GHz Connection, 1600 DPI | Type - C Adapter | Upto 12 Months of Battery Life | Ambidextrous Design | Suitable for PC/Mac/Laptop, (Expected Rating:4.66)
2, Sony Bravia 164 cm (65 inches) 4K Ultra HD Smart LED Google TV KD-65X74K (Black), (Expected Rating:4.57)
3, Syncwire LTG to USB Cable for Fast Charging Compatible with Phone 5/ 5C/ 5S/ 6/ 6S/ 7/8/ X/XR/XS Max/ 11/12/ 13 Series and Pad Air/Mini, Pod & Other Devices (1.1 Meter, White), (Expected Rating:4.55)
4, ESR Screen Protector Compatible with iPad Pro 11 Inch (2022/2021/2020/2018) and iPad Air 5/4 (2022/2020, 10.9 Inch), Tempered-Glass Film with Alignment Frame, Scratch Resistant, HD Clarity, 2 Pack, (Expected Rating:4.54)
5, Belkin USB C to USB-C Fast Charging Type C Cable, 60W PD, 3.3 feet (1 meter) for Laptop, Personal Computer, Tablet, Smartphone - White, USB-IF Certified, (Expected Rating:4.53)
6, Havells Aqua Plus 1.2 litre Double Wall Kettle / 304 Stainless Ste

In [149]:
print(recommendation())

Please input your id :  AE3MSW6H3AL6F3ZGR5LCN5AHJO6A


Top10 what you will love!
1, Amazon Basics Wireless Mouse | 2.4 GHz Connection, 1600 DPI | Type - C Adapter | Upto 12 Months of Battery Life | Ambidextrous Design | Suitable for PC/Mac/Laptop, (Expected Rating:4.56)
2, Syncwire LTG to USB Cable for Fast Charging Compatible with Phone 5/ 5C/ 5S/ 6/ 6S/ 7/8/ X/XR/XS Max/ 11/12/ 13 Series and Pad Air/Mini, Pod & Other Devices (1.1 Meter, White), (Expected Rating:4.53)
3, Instant Pot Air Fryer, Vortex 2QT, Touch Control Panel, 360° EvenCrisp™ Technology, Uses 95 % less Oil, 4-in-1 Appliance: Air Fry, Roast, Bake, Reheat (Vortex 1.97Litre, Black), (Expected Rating:4.49)
4, Oratech Coffee Frother electric, milk frother electric, coffee beater, cappuccino maker, Coffee Foamer, Mocktail Mixer, Coffee Foam Maker, coffee whisker electric, Froth Maker, coffee stirrers electric, coffee frothers, Coffee Blender, (6 Month Warranty) (Multicolour), (Expected Rating:4.47)
5, Sony Bravia 164 cm (65 inches) 4K Ultra HD Smart LED Google TV KD-65X74K (Blac

In [150]:
print(recommendation())

Please input your id :  1


You are not our member, Please sign in!
None
